In [ ]:
!pip install scikit-learn-extra

In [ ]:
!pip install keras-tcn

In [ ]:
! pip install rstl

In [ ]:
!pip install pmdarima

In [5]:
# !pip install neuralprophet

In [6]:
import pmdarima as pm

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from sklearn.cluster import SpectralClustering
# %tensorflow_version 1.x
import tensorflow as tf
import keras
import keras.backend as K
from keras import layers
from keras.models import Sequential,Model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import time
print(tf.__version__)
from keras.layers import MultiHeadAttention
from keras.layers import Dense
import gc
from keras.layers import concatenate
import csv
import math
import warnings
import os

# import xgboost as xgb
warnings.filterwarnings('ignore')
# import GPy, GPyOpt
tfkl = tf.keras.layers
tfk = tf.keras
from tcn import TCN

from rstl import STL
from texttable import Texttable

In [8]:
np.random.seed(1234)

In [ ]:
!pip install statsmodels

In [ ]:
!pip install statsforecast


In [11]:
from statsforecast.models import AutoARIMA


In [12]:
from statsmodels.tsa.api import SARIMAX, ARIMA,ExponentialSmoothing, SimpleExpSmoothing, Holt


In [13]:
# raw_data = pd.read_csv("/kaggle/input/nn5-dataset/Fixed_nn5.csv",_=',', header=None) # Hospital - Horizon 12
# features = pd.read_csv("/kaggle/input/nn5-featureset-autoformer/Features_nn5_Autoformer_max_8_label_mv41_raw.csv",sep=',', header=None) # CIF 2016 - Horizon 6

In [14]:

# print(raw_data.loc[0])

In [15]:
rd='Mahdian/'

def get_dataset_params(dataset_name = 'cif-12'):
    suilin_smape = False

    #-------------------------------------------------- CIF 2016 ------------------------------------------------#
    if dataset_name == 'cif-6':
        dataset_path = '/kaggle/working'+'/'+'CIF2016'
        raw_data = pd.read_excel("/kaggle/input/cifnewdataset/6.xlsx",header=None) # Kaggle - Wikipedia - Horizon 59
#         features = pd.read_csv("/kaggle/input/ae-cif012-unsupervised-ae/Features_LSTM_cif012_32.csv",sep=',', header=None) # CIF 2016 - Horizon 6
#         features = pd.read_csv('/kaggle/input/cif-data-and-hynd-khs-feature/fs_hyndman_freqfind_cif12.csv',sep=',', header=0) # CIF 2016 - Horizon 6

        lag = 15
        look_forward = 6
        batch_size = 3
        epochs = 30
        learning_rate = 0.0001
        features=raw_data

        suilin_smape = False
        frequency = None

    elif dataset_name == 'cif-12':
        dataset_path ='/kaggle/working'+'/'+ 'CIF2016'

        raw_data = pd.read_excel("/kaggle/input/cifnewdataset/12.xlsx",header=None) # Kaggle - Wikipedia - Horizon 59
#         features = pd.read_csv("/kaggle/input/ae-cif012-unsupervised-ae/Features_LSTM_cif012_32.csv",sep=',', header=None) # CIF 2016 - Horizon 6
        features = pd.read_csv('/kaggle/input/cif-data-and-hynd-khs-feature/fs_khs_CIF-12_with_find_freq.csv',sep=',', header=0) # CIF 2016 - Horizon 6
#         raw_data = raw_data.iloc[:, :-6]
        lag = 36
        look_forward = 12
        batch_size = 6
        epochs = 25
        learning_rate = 0.001

        suilin_smape = False
        frequency = 12
#         frequency = None


    #-------------------------------------------------- Hospital ------------------------------------------------#
    #-------------------------------------------------- Hospital ------------------------------------------------#

    elif dataset_name == 'tourism':
        dataset_path = rd+'/'+'tourism'
        raw_data = pd.read_excel("/kaggle/input/tourism/Tourism-new.xlsx",header=None) # Kaggle - Wikipedia - Horizon 59

        features = pd.read_csv('/kaggle/input/tourism/fs_KHS_Tourism.csv',sep=',', header=0) # CIF 2016 - Horizon 6
#         features = pd.read_csv('/kaggle/input/hospital-tourims-attention-feat/Features_tourism_attenLSTM_8.csv',sep=',', header=None) # CIF 2016 - Horizon 6


        lag = 20
        look_forward = 8
        batch_size = 100
        epochs =50
        learning_rate = 0.001
        suilin_smape = False
        frequency = 4
        # frequency = None      
    elif dataset_name == 'hospital':
        dataset_path = rd+'/'+'Hospital'
        raw_data = pd.read_excel("/kaggle/input/newtsdatasets/Hospital_new.xlsx",header=None) # Kaggle - Wikipedia - Horizon 59

#         raw_data = pd.read_csv("/kaggle/input/hospital-dataset/hospital_dataset.txt",sep='delimeter', header=None) # Hospital - Horizon 12
        features = pd.read_csv("/kaggle/input/hospital-dataset/fs_khs_hopsiptal_findfreq.csv",sep=',', header=0) # CIF 2016 - Horizon 6
#         features = pd.read_csv("/kaggle/input/ae-hospital-unsupervised-ae/Features_hospital_LSTM_32.csv",sep=',', header=None) # CIF 2016 - Horizon 6

        lag = 30
        look_forward = 12
        batch_size = 20
        epochs = 50
        learning_rate = 0.0001
#         frequency = 12
        frequency = None
    elif dataset_name == 'nn5':
        dataset_path = rd+'/'+'nn5'
        raw_data = pd.read_csv("/kaggle/input/nn5-dataset/nn51.csv",sep=',', header=None) # Hospital - Horizon 12
#         features = pd.read_csv("/kaggle/input/nn5-dataset/fs_hyndman_Fixed_nn5.csv",sep=',', header=0) # CIF 2016 - Horizo/kaggle/input/nn5-featureset-autoformer/Features_nn5_Autoformer_max_8_label_mv41_raw.csvn 6
        features = pd.read_csv("/kaggle/input/nn5-hyndman-khs/fs_KHS_NN5 (2).csv",sep=',', header=0) # CIF 2016 - Horizo/kaggle/input/nn5-featureset-autoformer/Features_nn5_Autoformer_max_8_label_mv41_raw.csvn 6

        #Features_nn5_Autoformer_max_mean_dim8_head8_mv41_25_stride5.
        lag = 150
        look_forward = 56
        batch_size = 20
        epochs = 50
        learning_rate = 0.0001
        frequency = 7
        # frequency = None

    #---------------------------------------------- Kaggle Web ---------------------------------------------------#

#     elif dataset_name == 'kaggle':
#         dataset_path = rd+'/'+'Kaggle'
#         raw_data = pd.read_csv("/kaggle/input/kaggle/kaggle_web_traffic_dataset.txt",sep='delimeter', header=None) # Kaggle - Wikipedia - Horizon 59
#         features = pd.read_csv("/kaggle/input/kaggle/kaggle_web_traffic_dataset-hyndman-features.csv",sep=',', header=0) # CIF 2016 - Horizon 6
# #         features = pd.read_csv("/kaggle/input/one-ae-kaggle-unsupervised-ae/Features_kaggle_8.csv",sep=',', header=None) # CIF 2016 - Horizon 6

#         lag = 400
#         look_forward = 59
#         batch_size = 70
#         epochs = 20
#         learning_rate = 0.0001
#         suilin_smape = True
#         frequency = 30
        # frequency = None

    #--------------------------------------------------M3 Monthly------------------------------------------------#
    elif dataset_name == 'm3-demo':
        dataset_path ='/kaggle/working'+'/'+ 'M3'
        raw_data = pd.read_excel("/kaggle/input/m3-att-lstm/m3-demo-new.xlsx",header=None) # Kaggle - Wikipedia - Horizon 59

        features = pd.read_csv("/kaggle/input/m3-hyndman-khs-features/fs_KHS_m3_demo.csv", sep=',', header=0)
#         features = pd.read_csv("/kaggle/input/m3-att-lstm/M3-attCNN/Features_demo_32.csv",sep=',', header=None)
        
        lag = 29
        look_forward = 18
        batch_size = 7
        epochs = 20
        learning_rate = 0.0001
        frequency = 12
#         frequency = None

    elif dataset_name == 'm3-finance':
        dataset_path ='/kaggle/working'+'/'+ 'M3'
        raw_data = pd.read_excel("/kaggle/input/m3-att-lstm/m3-finance-new.xlsx",header=None) # Kaggle - Wikipedia - Horizon 59
        features = pd.read_csv("/kaggle/input/m3-hyndman-khs-features/fs_KHS_m3_finance.csv",sep=',', header=0)
#         raw_data = raw_data.iloc[:, :-9]
#         features = pd.read_csv("/kaggle/input/m3-att-lstm/M3-attCNN/Features_finance_8.csv",sep=',', header=None)
        lag = 27
        look_forward = 18#18
        batch_size = 9
        epochs = 25
        learning_rate = 0.0001
        frequency = 12
#         frequency = None

    elif dataset_name == 'm3-industry':
        dataset_path ='/kaggle/working'+'/'+ 'M3'
        raw_data = pd.read_excel("/kaggle/input/m3-att-lstm/m3-industry-new.xlsx",header=None) # Kaggle - Wikipedia - Horizon 59
        #note
        features = pd.read_csv("/kaggle/input/m3-hyndman-khs-features/fs_KHS_m3_industry.csv",sep=',', header=0)
#         features = pd.read_csv("/kaggle/input/m3-att-lstm/M3-attCNN/Features_industry_32.csv",sep=',', header=None)

        lag = 26
        look_forward = 18
        batch_size = 10
        epochs = 25
        learning_rate = 0.0001
        frequency = 12
#         frequency = None

    elif dataset_name == 'm3-macro':
        dataset_path ='/kaggle/working'+'/'+ 'M3'
        raw_data = pd.read_excel("/kaggle/input/m3-att-lstm/m3-macro-new.xlsx",header=None) # Kaggle - Wikipedia - Horizon 59

#         features = pd.read_csv("/kaggle/input/m3-att-lstm/M3-attCNN/Features_macro_32.csv",sep=',', header=None)
        features = pd.read_csv("/kaggle/input/m3-hyndman-khs-features/fs_KHS_m3_macro.csv",sep=',', header=0) # CIF 2016 - Horizon 6

        lag = 26
        look_forward = 18
        batch_size = 15
        epochs = 25
        learning_rate = 0.0001
        frequency = 12
#         frequency = None

    elif dataset_name == 'm3-micro':
        dataset_path ='/kaggle/working'+'/'+ 'M3'
        raw_data = pd.read_excel("/kaggle/input/m3-att-lstm/M3-micro-new.xlsx",header=None) # Kaggle - Wikipedia - Horizon 59

        features = pd.read_csv("/kaggle/input/m3-hyndman-khs-features/fs_KHS_m3_micro.csv",sep=',', header=0)
#         features = pd.read_csv("/kaggle/input/m3-hyndman-khs-features/m3_micro-hyndman-features.csv",sep=',', header=0) # CIF 2016 - Horizon 6

        lag = 28
        look_forward = 18
        batch_size = 18
        epochs = 25
        learning_rate = 0.0001
#         frequency = 12
        frequency = None
# 
    elif dataset_name == 'm3-other':
        dataset_path ='/kaggle/working'+'/'+ 'M3'
        raw_data = pd.read_excel("/kaggle/input/m3-att-lstm/m3-other-new.xlsx",header=None) # Kaggle - Wikipedia - Horizon 59
#         features = pd.read_csv("/kaggle/input/m3-att-lstm/M3-attCNN/Features_other_32.csv",sep=',', header=None)
        features = pd.read_csv("/kaggle/input/m3-hyndman-khs-features/fs_KHS_m3_other.csv",sep=',', header=0) # CIF 2016 - Horizon 6

        lag = 26
        look_forward = 18
        batch_size = 2
        epochs = 25
        learning_rate = 0.0001
        frequency = 12
#         frequency = None

    #------------------------------------------------------------------------------------------------------------#

    sample_overlap = look_forward - 1

    raw_data = raw_data.to_numpy().astype('float64')
    features = features.to_numpy().astype('float64')
    dataset = []
    seri_len=[]
    for i in range(len(raw_data)):
        dataset.append(raw_data[i][~np.isnan(raw_data[i])])
        seri_len.append(len(raw_data[i][~np.isnan(raw_data[i])]))


    print(dataset_name,np.min(seri_len),np.max(seri_len))
        

    return dataset, features, lag, look_forward, sample_overlap, learning_rate, dataset_path, suilin_smape, frequency

In [16]:
def normalize_dataset(dataset, look_forward ):
    data_means = [];
    for index in range(len(dataset)):
    # Mean Noramlization
        series_mean = np.mean(dataset[index][:len(dataset[index]) - look_forward]) # Train Mean: look_forward || Full Mean: Mean: look_forward = 0

        if series_mean == 0:
            series_mean = 0.001

        data_means.append(series_mean)
        dataset[index] = np.divide(dataset[index], series_mean)
        if np.min(dataset[index][:len(dataset[index])])<=0:
                    dataset[index] = np.log(dataset[index] + 1)
        else:
             dataset[index] = np.log(dataset[index])
        # Log Transformation
#         dataset[index] = np.log(dataset[index] + 1)
       


    return dataset, np.array(data_means)

def rescale_data_to_main_value(data, means, dataset_seasonal = []):
    for index in range(len(data)):
        if len(dataset_seasonal) != 0:
            data[index] = data[index] + dataset_seasonal[index]
        # Revert Log Transformation
        data[index] = np.e ** data[index]
#         data[index] = data[index]

        # Revert Mean Normalization
        data[index] = means[index] * data[index]
    

    return data

def normalize_feature_vectors(features):
    #------------------- Z-score ----------------------#
#     means = features.mean(0)
#     stds = features.std(0)

#     for i in range(len(features)):
#         features[i] = (features[i] - means) / stds

    #--------------------Min - Max---------------------#
    minimum = features.min(0)
    maximum = features.max(0)

    for i in range(len(features)):
        features[i] = (features[i] - minimum) / (maximum - minimum)


    return features

"""![root_mean_square_deviation.svg](attachment:root_mean_square_deviation.svg)"""

#RMSE
def root_mean_squared_error(actual, forecast, method = 'single_value'):
    # Methods = single_value | per_series
    if method == 'single_value':
        #Flatten To One Vector
        actual = actual.flatten()
        forecast = forecast.flatten()

        return np.sqrt(np.mean(np.square(actual - forecast)))
    elif method == 'per_series':
        rmses = []
        for i in range(len(actual)):
            rmses.append(np.sqrt(np.mean(np.square(actual[i] - forecast[i]))))

        return rmses

"""![YIy33.png](attachment:YIy33.png)"""

#SMAPE
def single_point_smape(actual, forecast, suilin_smape = False):
    if suilin_smape == True:
        epsilon = 0.1

        return (np.sum(2 * np.abs(forecast - actual) / max((np.abs(actual) + np.abs(forecast))+ epsilon, 0.5 + epsilon)))
    else:
        return (np.sum(2 * np.abs(forecast - actual) / (np.abs(actual) + np.abs(forecast))))

def smape(actual, forecast, method = 'single_value', suilin_smape = False):
    # Methods = single_value | per_series
    if method == 'single_value':
        #Flatten To One Vector
        actual = actual.flatten()
        forecast = forecast.flatten()
        sum_smape = 0
        for i in range(len(actual)):
            sum_smape += single_point_smape(actual[i], forecast[i], suilin_smape)
        return 100 * sum_smape / len(actual)

    elif method == 'per_series':
        smapes = []
        for i in range(len(actual)):
            sum_smape = 0
            for j in range(len(actual[i])):
                sum_smape += single_point_smape(actual[i,j], forecast[i,j], suilin_smape)
            smapes.append(100 * sum_smape / len(actual[i]))
        return np.array(smapes)

# Create Samples from DataSet
def create_dataset(sample, look_back, look_forward, sample_overlap, dataset_seasonal):
    if(sample_overlap >= look_forward or sample_overlap < 0): sample_overlap = look_forward - 1
    if(look_forward == 1): sample_overlap = 0

    dataX, dataY, dataY_seasonal = [], [], []
    dataX_means, dataY_means = [], []
    for i in range(0, len(sample) - look_back - look_forward+1, look_forward - sample_overlap):
        dataX.append(sample[i:(i+look_back), 0])
        dataY.append(sample[(i + look_back):(i + look_back + look_forward), 0])

        dataY_seasonal.append(dataset_seasonal[(i + look_back):(i + look_back + look_forward)])


    return np.array(dataX), np.array(dataY), np.array(dataY_seasonal)

In [17]:
def create_dataset3_arima(sample, look_back, look_forward, sample_overlap, dataset_seasonal,dataset_name):
    if(sample_overlap >= look_forward or sample_overlap < 0): sample_overlap = look_forward - 1
    print('dddd')
    if(look_forward == 1): sample_overlap = 0
    # print("sample.shape",sample.shape)
    sample_trn=sample[0:len(sample)-look_forward]
    frequency=12
    if dataset_name=='tourism':
        frequency=4
    if dataset_name=='cif-6':
        frequency=12
#     print("freq",frequency)
    #         fit1 = pm.auto_arima(sample_trn,trace=True,error_action="ignore",stepwise=False,seasonal=True,m=frequency,information_criterion='aic')#,stepwise=True,information_criterion='aic')

    if len(sample_trn) > frequency*2:
        
        fit1 =pm.auto_arima(sample_trn, seasonal=True, m=12,maxiter=5,n_jobs=-1)
        aug_seri, conf_int = fit1.predict(n_periods=look_forward, return_conf_int=True)
        aug_seri = aug_seri.reshape(1,-1)
           
           
    elif len(sample_trn)<frequency*2 and len(sample_trn)>frequency :
        frequency = int(frequency/2)
        fit1 = pm.auto_arima(sample_trn, seasonal=True, m=12,maxiter=5)

        aug_seri, conf_int = fit1.predict(n_periods=look_forward, return_conf_int=True)
        aug_seri = aug_seri.reshape(1,-1)
    else:
        fit1 = pm.auto_arima(sample_trn, seasonal=True, m=12,maxiter=5)
        aug_seri, conf_int = fit1.predict(n_periods=look_forward, return_conf_int=True)
        aug_seri = aug_seri.reshape(1,-1)


#     fit1 = ExponentialSmoothing(endog=pd.Series(sample_trn.flatten()), seasonal_periods=12, trend='add',
#                                 seasonal='add').fit()
#     aug_seri = fit1.forecast(steps=look_forward).values.reshape(1,-1)
    # print(aug_seri)
    from_train = sample_trn[-(look_back+look_forward-1):]
    frm_train_aug=np.concatenate([from_train.reshape(-1,1),aug_seri.reshape(-1,1)])
    frm_train_aug=frm_train_aug.flatten()
    # print(frm_train_aug)
    # print(from_train[:,0])


    aug_trainX,aug_trainY=[],[]
    for i in range(0, len(frm_train_aug) - look_back - look_forward+1, look_forward - sample_overlap):
        aug_trainX.append(frm_train_aug[i:(i+look_back)])
        aug_trainY.append(frm_train_aug[(i + look_back):(i + look_back + look_forward)])





        # dataY_seasonal.append(dataset_seasonal[(i + look_back):(i + look_back + look_forward)])

    return np.array(aug_trainX), np.array(aug_trainY)

In [18]:
def create_dataset2(sample, look_back, look_forward, sample_overlap, dataset_seasonal,dataset_name):
    if(sample_overlap >= look_forward or sample_overlap < 0): sample_overlap = look_forward - 1
    if(look_forward == 1): sample_overlap = 0
    # print("sample.shape",sample.shape)
    sample_trn=sample[0:len(sample)-look_forward]
    
    frequency=12
    if dataset_name=='tourism':
        frequency=4
    if dataset_name=='cif-6':
        frequency=12
#     print("freq",frequency)
    
    if len(sample_trn) > frequency*2:
        fit1 = ExponentialSmoothing(endog=pd.Series(sample_trn.flatten()), seasonal_periods=frequency, trend='add',
                            seasonal='add').fit()
        aug_seri = fit1.forecast(steps=look_forward).values.reshape(1,-1)
           
           
    elif len(sample_trn)<frequency*2 and len(sample_trn)>frequency :
        frequency = int(frequency/2)
        fit1 = ExponentialSmoothing(endog=pd.Series(sample_trn.flatten()), seasonal_periods=frequency, trend='add',
                            seasonal='add').fit()
        aug_seri = fit1.forecast(steps=look_forward).values.reshape(1,-1)
    else:
        fit1 = ExponentialSmoothing(endog=pd.Series(sample_trn.flatten())).fit()

        aug_seri = fit1.forecast(steps=look_forward).values.reshape(1,-1)


#     fit1 = ExponentialSmoothing(endog=pd.Series(sample_trn.flatten()), seasonal_periods=12, trend='add',
#                                 seasonal='add').fit()
#     aug_seri = fit1.forecast(steps=look_forward).values.reshape(1,-1)
    # print(aug_seri)
    from_train = sample_trn[-(look_back+look_forward-1):]
    frm_train_aug=np.concatenate([from_train.reshape(-1,1),aug_seri.reshape(-1,1)])
    frm_train_aug=frm_train_aug.flatten()
    # print(frm_train_aug)
    # print(from_train[:,0])


    aug_trainX,aug_trainY=[],[]
    for i in range(0, len(frm_train_aug) - look_back - look_forward+1, look_forward - sample_overlap):
        aug_trainX.append(frm_train_aug[i:(i+look_back)])
        aug_trainY.append(frm_train_aug[(i + look_back):(i + look_back + look_forward)])





        # dataY_seasonal.append(dataset_seasonal[(i + look_back):(i + look_back + look_forward)])

    return np.array(aug_trainX), np.array(aug_trainY)

In [19]:
def stl_decomposition2(dataset, frequency,look):
    seasonal = []
    trend = []
    for index in range(len(dataset)):
        if frequency != None:
            stl = STL(dataset[index][:len(dataset[index]) - look], frequency, "periodic")
            stl_tot = STL(dataset[index], frequency, "periodic")

            seasonal.append(np.concatenate([stl.seasonal,stl_tot.seasonal[-look:]]))
            trend.append(np.concatenate([stl.trend,stl_tot.trend[-look:]]))
            # deseason_part=dataset[index][:len(dataset[index]) - look] - stl.seasonal
            # d=np.concatenate([deseason_part,dataset[index][-look:]])
            dataset[index]=dataset[index]-np.concatenate([stl.seasonal,stl_tot.seasonal[-look:]])
        else:
            seasonal.append(np.zeros((dataset[index].shape)))
            trend.append(np.zeros((dataset[index].shape)))

    return dataset, np.array(seasonal), np.array(trend)



In [20]:
#NOTE Frequency for MASE computing when no deseasonalization is happen

In [21]:
def single_point_mase(actual, forecast, insample, frequency) :
    if frequency==None:
        frequency=12
    # print("HHHHHHHHHHHHHHHHHHHHHHHHHHHH")
    return np.mean(np.abs(forecast - actual)) / np.mean(np.abs(insample[:-frequency] - insample[frequency:]))


def mase(actual, forecast, insample,frequency):
    if frequency==None:
        frequency=12
    print("shapes",actual.shape,forecast.shape,insample.shape)
    # print(insample)
    MASE = []
    for i in range(len(actual)):
        sum_MASE= 0
        for j in range(len(actual[i])):
            sum_MASE += single_point_mase(actual[i, j], forecast[i, j],insample[i][:-len(actual[i])], frequency)
        MASE.append(sum_MASE / len(actual[i]))
    return np.array(MASE)

In [22]:
def mase_val(actual, forecast, insample,frequency):
    if frequency==None:
        frequency=12
    print("shapes",actual.shape,forecast.shape,insample.shape)
    # print(insample)
    MASE = []
    for i in range(len(actual)):
        sum_MASE= 0
        for j in range(len(actual[i])):
            sum_MASE += single_point_mase(actual[i, j], forecast[i, j],insample[i][:-2*len(actual[i])], frequency)
        MASE.append(sum_MASE / len(actual[i]))
    return np.array(MASE)

In [23]:
def create_sample(look_forward,sample_seasonal,dataX, dataY, data_mean, dataY_seasonal,frequency):
    test_size=1
    val_size=1
    
    # print(frequency)
    # print(sample_seasonal)
    # test_size=int(len(dataX) * testsize)
    # val_size=int((len(dataX) - test_size) * valsize)
#     print(dataY)
    train_size0=(len(dataX)-test_size)-look_forward+1
    train_size=(len(dataX)-test_size)
#     print("train_size",train_size)

#     trainX, testX = dataX[0:train_size0,:], dataX[train_size:,:]  #
#     trainY, testY = dataY[0:train_size0,:], dataY[train_size:,:]


    trainX, testX = dataX[0:train_size0-val_size,:], dataX[train_size:,:]  #note -valsize is added now
    trainY, testY = dataY[0:train_size0-val_size,:], dataY[train_size:,:]

    valX, valY = dataX[train_size0-val_size:train_size0,:],dataY[train_size0-val_size:train_size0, :]
#     print("valY",valY)
    trainX = np.reshape(trainX, (trainX.shape[0],1, trainX.shape[1]))
    valX = np.reshape(valX, (valX.shape[0],1, valX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0],1, testX.shape[1]))

    val_means = np.full(len(valY), data_mean)
    test_means = np.full(len(testY), data_mean)

    val_seasonal = dataY_seasonal[train_size0-val_size:train_size0, :]
    # print("len val",len(val_seasonal))
    # print("val",val_seasonal)

    train=dataY_seasonal[:train_size0, :]
    train=train.reshape(-1,1)
#     train2=train[:len(train):len(valY[0])]
    # print(train2.reshape(1,-1))
    # print(len(train2))

    test2 = []


    # modl = pm.auto_arima(train2, trace=False,seasonal=True, stepwise=False, information_criterion='aic',)
    # preds = modl.predict(n_periods=len(valY[0]))
    # print("xx",train2.shape)
    sample_size = len(sample_seasonal.flatten()) - look_forward

    train3=sample_seasonal[:sample_size].flatten()
#     print("len,frequency",len(train3.flatten()),frequency)
    if frequency!=None:
        if len(train3.flatten()) > frequency*2:
            sp = frequency
           
            fit1 = ExponentialSmoothing(endog=pd.Series(train3.flatten()), seasonal_periods=sp,trend='add', seasonal='add').fit()

        elif len(train3.flatten())<frequency*2 and len(train3.flatten())>frequency :
            sp = int(frequency/2)
            fit1 = ExponentialSmoothing(endog=pd.Series(train3.flatten()), seasonal_periods=sp, trend='add',
                                        seasonal='add').fit()
        else:
            fit1 = ExponentialSmoothing(endog=pd.Series(train3.flatten())).fit()

        preds2 = fit1.forecast(steps=look_forward).values.reshape(1,-1)

        # print("preds",preds2,type(preds2))
    else:
        preds2=np.zeros(look_forward)
#     for i in range(0,len(val_seasonal[0])):
#         test2.append(val_seasonal[0][len(val_seasonal[0])-1])

    # print("datay_seasonal",dataY_seasonal)

    test_seasonal_y = dataY_seasonal[train_size:,:]
    # print("test",test_seasonal_y)
    # print("train2",train2.flatten())
    # np.savetxt('train.csv', train2.flatten(), delimiter=', ')

    return np.array(trainX),np.array(valX),np.array(testX),np.array(trainY),np.array(valY),np.array(testY), test_means, val_means, val_seasonal,test_seasonal_y, preds2

# Preprocess Data For Sampling
def all_pre_process(all_dataset, lag, look_forward, sample_overlap, data_means, dataset_seasonal,frequency):
    look_back = lag

    trainX = []
    trainY = []


    valX = []
    valY = []

    testX = []
    testY = []

    all_test_means = []
    all_val_means = []

    all_test_seasonals = []
    all_test_seasonals2 = []
    all_val_seasonals = []

    for index in range(len(all_dataset)):
        sample = np.array(all_dataset[index])
        sample = sample.reshape(sample.shape[0], 1)
        # sample_seasonal=np.array(dataset_seasonal[index])

        dataX_s, dataY_s, dataY_seasonal = create_dataset(sample, look_back, look_forward, sample_overlap, dataset_seasonal[index])

    
        temp_trainX, temp_valX, temp_testX, temp_trainY, temp_valY, temp_testY, test_means, val_means, val_seasonal, test_seasonal,test2 = create_sample(look_forward,dataset_seasonal[index],dataX_s,dataY_s,data_means[index], dataY_seasonal,frequency)

        trainX = trainX + temp_trainX.tolist()
        trainY = trainY + temp_trainY.tolist()
        #for adding augmented data
#         trainX = trainX + aug_trainX.tolist()
#         trainY = trainY + aug_trainY.tolist()

        valX = valX + temp_valX.tolist()
        valY = valY + temp_valY.tolist()

        testX = testX + temp_testX.tolist()
        testY = testY + temp_testY.tolist()

        all_test_means = all_test_means + test_means.tolist()
        all_val_means = all_val_means + val_means.tolist()
        all_test_seasonals = all_test_seasonals +  test_seasonal.tolist()
        all_test_seasonals2 = all_test_seasonals2 +test2.tolist()  # test_seasonal.tolist() #"NOTE"
        all_val_seasonals = all_val_seasonals + val_seasonal.tolist()


    return np.array(trainX), np.array(valX), np.array(testX), np.array(trainY), np.array(valY), np.array(testY), np.array(all_test_means), np.array(all_val_means), np.array(all_val_seasonals), np.array(all_test_seasonals),np.array(all_test_seasonals2)

def save_prediction_result(data, dataset_name = 'cif-6', dataset_path = ''):
    if dataset_name == '':
        filename = dataset_name + '-results.csv'
    else:
        filename = dataset_path + '/' + dataset_name + '-results.csv'


    df = pd.DataFrame(data)
    df.to_csv(filename, sep=',',index=False,header=False)

In [24]:
def all_pre_process_aug(all_dataset, lag, look_forward, sample_overlap, data_means, dataset_seasonal,frequency,dataset_name):
    look_back = lag

    trainX = []
    trainY = []
 

    valX = []
    valY = []

    testX = []
    testY = []

    all_test_means = []
    all_val_means = []

    all_test_seasonals = []
    all_test_seasonals2 = []
    all_val_seasonals = []

    for index in range(len(all_dataset)):
        sample = np.array(all_dataset[index])
        sample = sample.reshape(sample.shape[0], 1)
        # sample_seasonal=np.array(dataset_seasonal[index])

        dataX_s, dataY_s, dataY_seasonal = create_dataset(sample, look_back, look_forward, sample_overlap, dataset_seasonal[index])
        aug_trainX,aug_trainY = create_dataset2(sample, look_back, look_forward, sample_overlap, dataset_seasonal[index],dataset_name)
#         aug_trainX,aug_trainY = create_dataset3_arima(sample, look_back, look_forward, sample_overlap, dataset_seasonal[index],dataset_name)

        aug_trainX= np.reshape(aug_trainX, (aug_trainX.shape[0],1, aug_trainX.shape[1]))
    
        temp_trainX, temp_valX, temp_testX, temp_trainY, temp_valY, temp_testY, test_means, val_means, val_seasonal, test_seasonal,test2 = create_sample(look_forward,dataset_seasonal[index],dataX_s,dataY_s,data_means[index], dataY_seasonal,frequency)
        # print("temp_trainX.shape",temp_trainX.shape)
        trainX = trainX + temp_trainX.tolist()
        trainY = trainY + temp_trainY.tolist()
        #for adding augmented data
        trainX = trainX + aug_trainX.tolist()
        trainY = trainY + aug_trainY.tolist()

        valX = valX + temp_valX.tolist()
        valY = valY + temp_valY.tolist()

        testX = testX + temp_testX.tolist()
        testY = testY + temp_testY.tolist()

        all_test_means = all_test_means + test_means.tolist()
        all_val_means = all_val_means + val_means.tolist()
        all_test_seasonals = all_test_seasonals +  test_seasonal.tolist()
        all_test_seasonals2 = all_test_seasonals2 +test2.tolist()  # test_seasonal.tolist() #"NOTE"
        all_val_seasonals = all_val_seasonals + val_seasonal.tolist()


    return np.array(trainX), np.array(valX), np.array(testX), np.array(trainY), np.array(valY), np.array(testY), np.array(all_test_means), np.array(all_val_means), np.array(all_val_seasonals), np.array(all_test_seasonals),np.array(all_test_seasonals2)


In [25]:
def create_sample_sep_val(look_forward,sample_seasonal,dataX, dataY, data_mean, dataY_seasonal,frequency):
    test_size=1
    val_size=1
    # print(frequency)
    # print(sample_seasonal)
    # test_size=int(len(dataX) * testsize)
    # val_size=int((len(dataX) - test_size) * valsize)
#     print("LENNNN",len(dataX))
    train_size=(len(dataX)-test_size-val_size)
#     print("trainsize, train+val",train_size,train_size +val_size)

    trainX, testX = dataX[0:train_size,:], dataX[train_size+val_size:,:]
    trainY, testY = dataY[0:train_size,:], dataY[train_size+val_size:,:]

    valX, valY = dataX[train_size:train_size +val_size,:],dataY[train_size:train_size+val_size, :]
#     print("LENNNN",len(valX),len(valY),valX.shape,valY.shape)
    trainX = np.reshape(trainX, (trainX.shape[0],1, trainX.shape[1]))
    valX = np.reshape(valX, (valX.shape[0],1, valX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0],1, testX.shape[1]))

    val_means = np.full(len(valY), data_mean)
    test_means = np.full(len(testY), data_mean)

    val_seasonal = dataY_seasonal[train_size:train_size+val_size, :]
    # print("len val",len(val_seasonal))
#     print("valseasonal ",val_seasonal.shape)

#     train=dataY_seasonal[:train_size, :]
#     train=train.reshape(-1,1)
   
    # print(train2.reshape(1,-1))
    # print(len(train2))

#     test2 = []


    # modl = pm.auto_arima(train2, trace=False,seasonal=True, stepwise=False, information_criterion='aic',)
    # preds = modl.predict(n_periods=len(valY[0]))
    # print("xx",train2.shape)
    sample_size = len(sample_seasonal.flatten()) - look_forward

    train3=sample_seasonal[:sample_size].flatten()

    if frequency!=None:
        if len(train3.flatten()) > frequency*2:
            sp = frequency
            fit1 = ExponentialSmoothing(endog=pd.Series(train3.flatten()), seasonal_periods=sp,trend='add', seasonal='add').fit()

        elif len(train3.flatten())<frequency*2 and len(train3.flatten())>frequency :
            sp = int(frequency/2)
            fit1 = ExponentialSmoothing(endog=pd.Series(train3.flatten()), seasonal_periods=sp, trend='add',
                                        seasonal='add').fit()
        else:
            fit1 = ExponentialSmoothing(endog=pd.Series(train3.flatten())).fit()

        preds2 = fit1.forecast(steps=look_forward).values.reshape(1,-1)

        # print("preds",preds2,type(preds2))
    else:
        preds2=np.zeros(look_forward)
#     for i in range(0,len(val_seasonal[0])):
#         test2.append(val_seasonal[0][len(val_seasonal[0])-1])

    # print("datay_seasonal",dataY_seasonal)

    test_seasonal_y = dataY_seasonal[train_size+val_size:,:]
    
#     print("test_seasonal",test_seasonal_y.shape)
    # print("train2",train2.flatten())
    # np.savetxt('train.csv', train2.flatten(), delimiter=', ')

    return np.array(trainX),np.array(valX),np.array(testX),np.array(trainY),np.array(valY),np.array(testY), test_means, val_means, val_seasonal,test_seasonal_y, preds2




In [26]:
#ss

In [27]:
def creat_model_lstm(lag,dense_neuron,look_forward):
    print('LSTM')
    denselayer_activation = 'relu' #None
    output_activation = 'linear' #'linear'
    input_layer = layers.Input(shape = (1, lag,), name = "Input-Layer")

    lstm1=keras.layers.LSTM(16,activation='relu',return_sequences=True)(input_layer)
    lstm2=keras.layers.LSTM(8,activation='relu',return_sequences=True)(lstm1)
    conv = keras.layers.Conv1D(4,
                                  strides=2,
                                  kernel_size=4,
                                  activation=None,
                                  padding="same",)(lstm2)

    flatten_layer2=keras.layers.Flatten(name="Flatten-Layer2")(conv)

    dense_layer1 = Dense(
        look_forward,
        activation = denselayer_activation, 
   

        name = "Fully-Connected-Layer")(flatten_layer2)

    dense_layer2 = Dense(
        look_forward,
        activation = None,
        name = "Output-Layer")(dense_layer1)

    # Create Model
    model = Model(inputs = [input_layer], outputs = dense_layer2)
    return model

In [28]:
def creat_model_TFR(lag,dense_neuron,look_forward):
    dense_neuron=50
    denselayer_activation = 'linear'#'linear' #None
    output_activation = 'linear'#'linear' #'linear'
    input_layer = layers.Input(shape = (1, lag,), name = "Input-Layer")
#     multi_head_attention_layer = TCN(return_sequences=True,dilations=[1, 2, 4, 8,16])(input_layer)
    multi_head_attention_layer = MultiHeadAttention(
    num_heads = lag,
    key_dim = 1,
    name="Main-Layer")(input_layer,input_layer,input_layer)
    glm=keras.layers.GlobalMaxPool1D()(multi_head_attention_layer)

    # LSTM_layer2 = layers.LSTM(units=50,activation=None,return_sequences=True)(LSTM_layer)

    flatten_layer1 = keras.layers.Flatten(name="Flatten-Layer")(glm)
    flatInput=keras.layers.Flatten(name="Flatten-Layer2")(input_layer)
    concat=keras.layers.concatenate([flatten_layer1,flatInput])

    dense_layer1 = Dense(
    dense_neuron,
    activation = denselayer_activation,
    name = "Fully-Connected-Layer")(concat)
    drp=keras.layers.Dropout(0)(dense_layer1)

    dense_layer2 = Dense(
    look_forward,
    activation = None,
    name = "Output-Layer")(drp)
#     drp=keras.layers.Dropout(0.1)(dense_layer1)



        # Create Model
    model = Model(inputs = [input_layer], outputs = dense_layer2)
    return model

In [29]:
def creat_model_tcn(lag,dense_neuron,look_forward):
    denselayer_activation = 'linear' #None
    output_activation = 'linear' #'linear'
    input_layer = layers.Input(shape = (1, lag,), name = "Input-Layer")
    multi_head_attention_layer = TCN(return_sequences=True,dilations=[1, 2, 4, 8])(input_layer)
    conv = keras.layers.Conv1D(64,
                      strides=2,
                      kernel_size=4,
                      activation=None,
                      padding="same",)(multi_head_attention_layer)#multi_head_attention_layer
    conv2 = keras.layers.Conv1D(16,
                      strides=2,
                      kernel_size=4,
                      activation=None,
                      padding="same",)(conv)
    flatten_layer2=keras.layers.Flatten(name="Flatten-Layer2")(conv2)
    # concat=keras.layers.concatenate([flatten_layer1,flatten_layer2])

    dense_layer1 = Dense(
            look_forward,
            activation = denselayer_activation,
            name = "Fully-Connected-Layer")(flatten_layer2)

    dense_layer2 = Dense(
            look_forward,
            activation = None,
            name = "Output-Layer")(dense_layer1)

        # Create Model
    model = Model(inputs = [input_layer], outputs = dense_layer2)
    return model

In [30]:
#  callback = keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)
# history = model.fit([trainX], trainY, validation_data=([valX, valY]),
#                     verbose = 0,
#                     batch_size = batch_size, epochs=epochs,callbacks=[callback]).history

In [31]:

def run_model_test(dataset, data_means, dataset_seasonal, dataset_name, cluster_lable, lag, look_forward, sample_overlap, batch_size, epochs, learning_rate, suilin_smape, dataset_path,frequency, use_saved_model = False, save_trained_model = False):
    print("len dataset",len(dataset))
    # Initialize Look Forward & Back
    look_back=lag
    calculations_method = 'per_series' # single_value | per_series
    # Loop Trough Clusters
    pre_start_time = time.time()
#     trainX, valX, testX, trainY, valY, testY, test_means, val_means, val_seasonal, test_seasonal,test_seasonal2 = all_pre_process(dataset, lag, look_forward, sample_overlap, data_means, dataset_seasonal,frequency)
    trainX, valX, testX, trainY, valY, testY, test_means, val_means, val_seasonal, test_seasonal,test_seasonal2 = all_pre_process_aug(dataset, lag, look_forward, sample_overlap, data_means, dataset_seasonal,frequency,dataset_name)
    pre_end_time = time.time()
    pre_execution_time = pre_end_time - pre_start_time
    print(f"Total Preprocessing time Execution time dataset {dataset_name}: {pre_execution_time} seconds")


    # 

    if use_saved_model == True:
        if os.path.exists(dataset_path + '/' + dataset_name + '-model-cluster-' + str(cluster_lable)) == True:
            model = keras.models.load_model(dataset_path + '/' + dataset_name  + '-model-cluster-' + str(cluster_lable))

            val_prediction_results = model.predict([valX],batch_size=16, verbose=0)

            val_RMSE = root_mean_squared_error(valY, val_prediction_results, calculations_method)
            val_SMAPE = smape(valY, val_prediction_results, calculations_method, suilin_smape)

            ######################################################
            test_prediction_results = model.predict([testX],batch_size=16, verbose=0)

            test_RMSE = root_mean_squared_error(testY, test_prediction_results, calculations_method)
            test_SMAPE = smape(testY, test_prediction_results, calculations_method, suilin_smape)

        else:
            use_saved_model = False
            save_trained_model = True

    # Train Model From Scratch
    if use_saved_model == False:

        dense_neuron = 100 ###JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ
        denselayer_activation = 'linear' #None
        output_activation = 'linear' #'linear'

        print("---------------------------------------------------------------------")
        print("lag", lag)
        print("look_forward", look_forward)
        print("sample overlap", sample_overlap)
        print("trainshape", trainX.shape)
        print("valshape", valX.shape)
        print("testshape", testX.shape)
        print(learning_rate, dense_neuron, denselayer_activation, output_activation)



        validation_loss=[]
        test_loss=[]
        iter = 1
        train_start_time = time.time()                              
        for j in range(iter):
            model=creat_model_tcn(lag,dense_neuron,look_forward)
         
            # Optimizer
            opt=tf.keras.optimizers.Adam(learning_rate=learning_rate)

            # Compile Model
            model.compile(loss="mse",optimizer=opt,metrics=["mse"])


            for k in range(epochs): #epochs
                history = model.fit([trainX], trainY, validation_data=([valX, valY]),verbose = 0,
                                    batch_size = batch_size).history

                val_prediction_results = model.predict([valX],batch_size=16, verbose=0)

                val_RMSE = root_mean_squared_error(valY, val_prediction_results, calculations_method)
                val_SMAPE = smape(valY, val_prediction_results, calculations_method, suilin_smape)

                ######################################################
                test_prediction_results = model.predict([testX],batch_size=16, verbose=0)

                test_RMSE = root_mean_squared_error(testY, test_prediction_results, calculations_method)
                test_SMAPE = smape(testY, test_prediction_results, calculations_method, suilin_smape)

                validation_loss.append(np.mean(val_RMSE))
                test_loss.append(np.mean(test_RMSE))

            K.clear_session()
            gc.collect()

            # Save model to a file if wanted
            if save_trained_model == True:
                model.save(dataset_path + '/' + dataset_name + '-model-cluster-' + str(cluster_lable))

            model=None
            del history
        train_end_time = time.time()
    train_execution_time = train_end_time - train_start_time
    print(f"Total Training model time Execution time dataset {dataset_name}: {train_execution_time} seconds")
    #############################################
    #############################################
    print(test_seasonal.shape)
    print("ddddddddddddddddd")
    print(test_seasonal2.shape)
    rescaled_valY = rescale_data_to_main_value(valY, val_means, val_seasonal)
#     print(" rescaled_valY", rescaled_valY)
    rescaled_val_prediction_results = rescale_data_to_main_value(val_prediction_results, val_means,val_seasonal) #####
    val_SMAPE = smape(rescaled_valY, rescaled_val_prediction_results, calculations_method, suilin_smape)
    val_RMSE = root_mean_squared_error(rescaled_valY, rescaled_val_prediction_results, calculations_method)
    # print("test seasonal",test_seasonal)
    # print("val",val_seasonal)
    # print("val[-1]",val_seasonal[-1])
    rescaled_testY = rescale_data_to_main_value(testY, test_means, test_seasonal)
#     print(" rescaled_testY", rescaled_testY)

    rescaled_trainY = rescale_data_to_main_value(dataset, data_means, dataset_seasonal)
    val_MASE = mase_val(rescaled_valY,rescaled_val_prediction_results,rescaled_trainY,frequency)
    # print("via resc",test_seasonal)
    rescaled_test_prediction_results = rescale_data_to_main_value(test_prediction_results, test_means,test_seasonal2 ) ###,test_seasonal
    test_MASE = mase(rescaled_testY,rescaled_test_prediction_results,rescaled_trainY,frequency)
    test_SMAPE = smape(rescaled_testY, rescaled_test_prediction_results, calculations_method)
    test_RMSE = root_mean_squared_error(rescaled_testY, rescaled_test_prediction_results, calculations_method)




    # print("TestY",rescaled_testY)
    # print( "Prediction",rescaled_test_prediction_results)


    results = {
            'val_SMAPE': val_SMAPE,
            'val_RMSE': val_RMSE,
            'val_MASE': val_MASE,
            'test_SMAPE': test_SMAPE,
            'test_RMSE': test_RMSE,
            'test_MASE': test_MASE
        }
    #############################################

    return results



In [32]:

clustering_results=[]

In [33]:
from sklearn.metrics import silhouette_score

def cluster_series(features, number_of_clusters=2):
    global clustering_results
    if len(clustering_results)==0:
#         clustered = KMedoids(n_clusters=number_of_clusters,init='k-medoids++',random_state=0).fit(features) # Kmedoids
#         print("Kmedoids")
#         clustered = KMeans(n_clusters=number_of_clusters,init='k-means++', random_state=0).fit(features) # Kmeans
#         print("Kmeans")
# #         clustering_results=clustered.labels_
        cluster_start_time = time.time()
        clustered = SpectralClustering(n_clusters=number_of_clusters,assign_labels = 'discretize',
                                      random_state = 0).fit(features)
        print("Spectral clustering")
        clustering_results=clustered.labels_

        cluster_end_time = time.time()
        culster_execution_time = cluster_end_time - cluster_start_time
        print(f"Clustering time Execution time dataset {dataset_name}: {culster_execution_time} seconds")
    # clustered = KMeans(n_clusters=number_of_clusters, random_state=0).fit(features) # Kmeans
    # print("Kmeans")

        print('silhouette_score -------->', silhouette_score(features, clustered.labels_))
        return clustered.labels_
    else:
        print('Using Saved clustering_results' )
        return clustering_results
        

def stl_decomposition(dataset, frequency):
    seasonal = []
    trend = []
    for index in range(len(dataset)):
        if frequency != None:
            stl = STL(dataset[index], frequency, "periodic")

            seasonal.append(stl.seasonal)
            trend.append(stl.trend)
            dataset[index] = dataset[index] - stl.seasonal
        else:
            seasonal.append(np.zeros((dataset[index].shape)))
            trend.append(np.zeros((dataset[index].shape)))

    return dataset, np.array(seasonal), np.array(trend)



In [34]:
def run_local_models(dataset_name, number_of_clusters=2, AEName='LSTM', Dim=8, epochs = 20, batch =20, use_saved_model = False, save_trained_model = False, run=1):
    import gc
    gc.collect()
    print('dataset: ', dataset_name)
    batch_size = batch
    epochs = epochs
    # Prepare & Read Data
    dataset, features, lag, look_forward, sample_overlap, learning_rate, dataset_path, suilin_smape, frequency = get_dataset_params(dataset_name)

    # Normalize Data
    dataset, data_means = normalize_dataset(dataset, look_forward)

    dataset, seasonal, trend = stl_decomposition2(dataset, frequency,look_forward)

    # Normalize Features
    features = normalize_feature_vectors(features)

    # Cluster Series Based On Feature Vectors (Feature Based Clustering)
    if number_of_clusters == 1:
        clusters = np.zeros(len(features))
    else:
        clusters = cluster_series(features, number_of_clusters)

    dataset = np.array(dataset)

    results = {
        'val_SMAPE': np.array([]),
        'val_RMSE': np.array([]),
        'val_MASE': np.array([]),
        'test_SMAPE': np.array([]),
        'test_RMSE': np.array([]),
        'test_MASE': np.array([])
    }

    # Loop Trough Clusters
    start_time = time.time()
    for cluster_lable in range(number_of_clusters):
        idx = [x for x in range(len(clusters)) if clusters[x] == cluster_lable]
        cluster_dataset = np.array(dataset)[idx]
        cluster_dataset_means = data_means[idx]
        cluster_dataset_seasonal = seasonal[idx]

        result = run_model_test(cluster_dataset, cluster_dataset_means, cluster_dataset_seasonal, dataset_name, cluster_lable, lag, look_forward, sample_overlap, batch_size, epochs, learning_rate, suilin_smape, dataset_path, frequency, use_saved_model, save_trained_model)

        results = {
            'val_SMAPE': np.concatenate((results['val_SMAPE'], result['val_SMAPE'])),
            'val_RMSE': np.concatenate((results['val_RMSE'], result['val_RMSE'])),
            'val_MASE': np.concatenate((results['val_MASE'], result['val_MASE'])),
            'test_SMAPE': np.concatenate((results['test_SMAPE'], result['test_SMAPE'])),
            'test_RMSE': np.concatenate((results['test_RMSE'], result['test_RMSE'])),
            'test_MASE': np.concatenate((results['test_MASE'], result['test_MASE']))
        }
        
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Total Cluster Training time Execution time dataset {dataset_name}: {execution_time} seconds")
    # print("test smape",results['test_SMAPE'])
    # Print Results Table
    filename_val_smape = "val_SMAPE_"+str(run)+"_"+dataset_name+"_"+str(batch_size)+"_"+str(epochs)+".npy"
    filename_val_mase = "val_MASE_"+str(run)+"_"+dataset_name+"_"+str(batch_size)+"_"+str(epochs)+".npy"
    filename_val_rmse = "val_RMSE_"+str(run)+"_"+dataset_name+"_"+str(batch_size)+"_"+str(epochs)+".npy"
    
    filename_test_smape = "test_SMAPE_"+str(run)+"_"+dataset_name+"_"+str(batch_size)+"_"+str(epochs)+".npy"
    filename_test_mase = "test_MASE_"+str(run)+"_"+dataset_name+"_"+str(batch_size)+"_"+str(epochs)+".npy"

    filename_test_rmse = "test_RMSE_"+str(run)+"_"+dataset_name+"_"+str(batch_size)+"_"+str(epochs)+".npy"

    # Save the NumPy array to the file
    np.save(filename_val_smape, results['val_SMAPE'])
    np.save(filename_val_rmse, results['val_RMSE'])
    np.save(filename_val_mase, results['val_MASE'])

    np.save(filename_test_smape, results['test_SMAPE'])
    np.save(filename_test_rmse, results['test_RMSE'])
    np.save(filename_test_mase, results['test_MASE'])
    t = Texttable()
    print('\n\n#------------------------------------Scaled------------------------------------#')
    t.add_rows([
        ['Index', 'Mean sMAPE', 'Median sMAPE', 'Mean RMSE', 'Median RMSE', 'Mean MASE'],
        ['Validate', np.mean(results['val_SMAPE']), np.median(results['val_SMAPE']), np.mean(results['val_RMSE']), np.median(results['val_RMSE']), np.mean(results['val_MASE'])],
        ['Test', np.mean(results['test_SMAPE']), np.median(results['test_SMAPE']), np.mean(results['test_RMSE']), np.median(results['test_RMSE']), np.mean(results['test_MASE'])]
    ])
    
    print(t.draw())
    t = Texttable()
    t.add_rows([
        ['dataset Name', 'Run', 'N.Epochs', 'Batch Size', 'Auto Encoder', 'Latent Dimension'],
        [dataset_name, run, epochs,batch_size, AEName, Dim]
    ])
    print(t.draw())
    initial_data = [
    ['dataset Name', 'Run', 'N.Epochs', 'Batch Size', 'Auto Encoder', 'Latent Dimension', 'Index', 'Mean sMAPE', 'Median sMAPE', 'Mean RMSE', 'Median RMSE', 'Mean MASE','Median MASE'],
    [dataset_name, run, epochs, batch_size, AEName, Dim, 'Validate', np.mean(results['val_SMAPE']), np.median(results['val_SMAPE']), np.mean(results['val_RMSE']), np.median(results['val_RMSE']), np.mean(results['val_MASE']),np.median(results['val_MASE'])],
    [dataset_name, run, epochs, batch_size, AEName, Dim, 'Test', np.mean(results['test_SMAPE']), np.median(results['test_SMAPE']), np.mean(results['test_RMSE']), np.median(results['test_RMSE']), np.mean(results['test_MASE']),np.median(results['test_MASE'])]
    ]# Write the header and initial data to the CSV file

    return results, initial_data

In [35]:
five_run= {
        'mean_val_SMAPE_5': np.array([]),
        'mean_val_RMSE_5': np.array([]),
        'mean_test_SMAPE_5': np.array([]),
        'mean_test_RMSE_5': np.array([]),
        'median_val_SMAPE_5': np.array([]),
        'median_val_RMSE_5': np.array([]),
        'median_test_SMAPE_5': np.array([]),
        'median_test_RMSE_5': np.array([])
    }









# ###########################
# ##############################
def cif(AEName="LSTM", Dim=16, run=1):
#     ds_names = [AEName, [5,15,25,35,50], [3,6,10, 20,40,60,80,100], Dim]
    ds_names = [AEName, [50], [6], Dim]

    file_mode = 'w'
    csv_file = AEName+"_"+str(Dim)+".csv"
    file_list.append(csv_file)
    if os.path.exists(csv_file):
        file_mode = 'a'
    else:
        file_mode = 'w'
    with open(csv_file, mode=file_mode, newline='') as file:
        writer = csv.writer(file)
        for epoch in ds_names[1]:
            for batch in ds_names[2]:
                    cif12_results, initial_data_cif12 = run_local_models(dataset_name = 'cif-12', number_of_clusters = 2, AEName=ds_names[0], Dim=ds_names[3], epochs = epoch, batch = batch, use_saved_model = False, save_trained_model = False, run=run)
                    print('\n----------------------------------------------------------------------------------------------\n')
                    cif6_results, initial_data_cif06 = run_local_models(dataset_name = 'cif-6', number_of_clusters = 1, AEName=ds_names[0], Dim=ds_names[3], epochs = epoch, batch = batch, use_saved_model = False, save_trained_model = False, run=run)
                    print('\n----------------------------------------------------------------------------------------------\n')
                    print('\n--------------------------------------Mixed Results---------------------------------------------\n')
                    results = {
                        'val_SMAPE': np.concatenate((cif12_results['val_SMAPE'], cif6_results['val_SMAPE'])),
                        'val_RMSE': np.concatenate((cif12_results['val_RMSE'], cif6_results['val_RMSE'])),
                        'val_MASE': np.concatenate((cif12_results['val_MASE'], cif6_results['val_MASE'])),
                        'test_SMAPE': np.concatenate((cif12_results['test_SMAPE'], cif6_results['test_SMAPE'])),
                        'test_RMSE': np.concatenate((cif12_results['test_RMSE'], cif6_results['test_RMSE'])),
                        'test_MASE': np.concatenate((cif12_results['test_MASE'], cif6_results['test_MASE']))
                    }

                    t = Texttable()
                    print('\n\n#------------------------------------Scaled------------------------------------#')
                    t.add_rows([
                        ['Index', 'Mean sMAPE', 'Median sMAPE', 'Mean RMSE', 'Median RMSE', 'Mean MASE','Meadian MASE'],
                        ['Validate', np.mean(results['val_SMAPE']), np.median(results['val_SMAPE']), np.mean(results['val_RMSE']), np.median(results['val_RMSE']),np.mean(results['val_MASE']), np.median(results['val_MASE'])],
                        ['Test', np.mean(results['test_SMAPE']), np.median(results['test_SMAPE']), np.mean(results['test_RMSE']), np.median(results['test_RMSE']),np.mean(results['test_MASE']), np.median(results['test_MASE'])]
                    ])
                    initial_data = []
                    
                    initial_data = initial_data_cif12 + initial_data_cif06
                    initial_data.extend(
                        [
                        ['dataset Name', 'Run', 'N.Epochs', 'Batch Size', 'Auto Encoder', 'Latent Dimension', 'Index', 'Mean sMAPE', 'Median sMAPE', 'Mean RMSE', 'Median RMSE','Mean MASE', 'Median MASE'],
                        ['Mixed', run, epoch, batch, AEName, Dim, 'Validate', np.mean(results['val_SMAPE']), np.median(results['val_SMAPE']), np.mean(results['val_RMSE']), np.median(results['val_RMSE']),np.mean(results['val_MASE']), np.median(results['val_MASE'])],
                        ['Mixed', run, epoch, batch, AEName, Dim, 'Test', np.mean(results['test_SMAPE']), np.median(results['test_SMAPE']), np.mean(results['test_RMSE']), np.median(results['test_RMSE']),np.mean(results['test_MASE']), np.median(results['test_MASE'])]
                        ]
                    )
                    writer.writerows(initial_data)
                    print(t.draw())

In [36]:
# /kaggle/input/nn5-autoformer-testing-features/Features_nn5_Autoformer_max_mean_8_label_mv41_raw_head8_e10_mvg25.csv

In [37]:
file_list=[]

In [38]:
#mdll
num_cluster2={'tourism':1,'cif-12':1,'m3-demo':1, 'm3-finance':1, 'm3-industry':1, 'm3-micro':1, 'm3-macro':1, 'm3-other':1, 'nn5':1, 'hospital':1 }
def get_param2():
    dataset_name='hospital'#'m3-demo'
    num_clust=num_cluster2[dataset_name]
#     num_clust=1

    batch=[10]
    epoch=[10]# 5,10,15

#     epoch=[5,10,15,25,35,50]# 5,10,15

#     batch=[2,3,4,5,6,7,10]
#     epoch=[5,10,20,30]
#     batch=[3]
#     epoch=[15]
#     batch=[10,20,30,35,40,60,80]
#     epoch=[5,10,15,20,30,40,50]# 5,10,15

    mdl=dataset_name+"LSTM_8"
    
    return dataset_name,num_clust,batch, epoch,mdl

In [39]:
#USED FOR ALL DATASET EXCEPT CIF CIF CIF CIF CIF CIF CIF CIF
def m3(AEName="", Dim=0, run=1):
#     ds_names = [AEName, [5, 15, 25, 35, 50], [20, 40, 60, 80, 100], Dim]
#     ds_names = [AEName, [1,50], [20,10000], Dim]
    dataset_name,num_cluster,batch, epoch,mdl=get_param2()
    ds_names = [AEName, epoch, batch, Dim]
    file_mode = 'w'
    csv_file = AEName+"_"+str(Dim)+".csv"
   
    file_list.append(csv_file)
    file_mode = 'w'


    if os.path.exists(csv_file):
        file_mode = 'a'
    else:
        file_mode = 'w'
    with open(csv_file, mode=file_mode, newline='') as file:
        writer = csv.writer(file)
        for epoch in ds_names[1]:
            for batch in ds_names[2]:
                if epoch==500 and batch==200:
                    continue
                else:
                    nn5_results, initial_data_nn5 = run_local_models(dataset_name =dataset_name , number_of_clusters = num_cluster, AEName=ds_names[0], Dim=ds_names[3], epochs = epoch, batch = batch, use_saved_model = False, save_trained_model = False, run=run)
                    print('\n----------------------------------------------------------------------------------------------\n')
                    results = {
                        'val_SMAPE': nn5_results['val_SMAPE'],
                        'val_RMSE': nn5_results['val_RMSE'],
                        'val_MASE': nn5_results['val_MASE'],
                        'test_SMAPE': nn5_results['test_SMAPE'],
                        'test_RMSE':nn5_results['test_RMSE'] ,
                        'test_MASE':nn5_results['test_MASE']
                    }
                    t = Texttable()
                    print('\n\n#------------------------------------Scaled------------------------------------#')
                    t.add_rows([
                        ['Index', 'Mean sMAPE', 'Median sMAPE', 'Mean RMSE', 'Median RMSE','Mean MASE', 'Median MASE'],
                        ['Validate', np.mean(results['val_SMAPE']), np.median(results['val_SMAPE']), np.mean(results['val_RMSE']), np.median(results['val_RMSE']),np.mean(results['val_MASE']), np.median(results['val_MASE'])],
                        ['Test', np.mean(results['test_SMAPE']), np.median(results['test_SMAPE']), np.mean(results['test_RMSE']), np.median(results['test_RMSE']),np.mean(results['test_MASE']), np.median(results['test_MASE'])]
                    ])
                    initial_data = []

                    initial_data = initial_data_nn5 

                    writer.writerows(initial_data)
                    print(t.draw())

In [40]:
#NOTTTTTTTTTTTTTfor i in range(2):

In [ ]:
for i in range(1):
    dataset_name,num_cluster,batch, epoch,mdl=get_param2()
    dim=0
    m3(mdl,dim,run=i)
#     cif(mdl,dim,run=i)

        


# t1 = Texttable()
# print('\n\n#------------------------------------Scaled------------------------------------#')
# t1.add_rows([
#     ['Index', 'Mean sMAPE', 'Median sMAPE', 'Mean RMSE', 'Median RMSE'],
#     ['Validate', five_run['mean_val_SMAPE_5'], five_run['mean_val_RMSE_5'], five_run['median_val_SMAPE_5'], five_run['median_val_RMSE_5']],
#     ['Test', five_run['mean_test_SMAPE_5'], five_run['mean_test_RMSE_5'], five_run['median_test_SMAPE_5'], five_run['median_test_RMSE_5']]
# ])
# print(t1.draw())

In [ ]:
file_list

In [ ]:
import os
import pandas as pd
import warnings

warnings.simplefilter(action='ignore', category=Warning)

directory_path = '/kaggle/working/'
# csv_file='results_Hyndman_0.csv'
# file_list = os.listdir(directory_path)

# csv_files = [csv_file]
# file_list=['LSTM_results_hospital_LSTM_8_32.csv']
# Iterate through CSV files and process them
for csv_file in file_list[-1:]:
    file_path = os.path.join(directory_path, csv_file)
    df = pd.read_csv(file_path, header=None)
    print(df)

    results = []

#     epochs = ['5', '15', '25', '35', '50']
#     batchs = ['20', '40', '60', '80', '100']
    dataset_name,num_cluster,batchs, epochs,mdl=get_param2()
    print("hhhhhhhhhhhhhhh")

  

    for epoch in epochs:
        for batch in batchs:
            if epoch==500 and batch==200:
                continue
            validate_df = df[(df[2] == str(epoch)) & (df[3] == str(batch)) & (df[6] == 'Validate')]
            test_df = df[(df[2] == str(epoch)) & (df[3] == str(batch)) & (df[6] == 'Test')]
            validate_df.loc[:, [7,8, 9, 10,11,12]] = validate_df.loc[:, [7,8, 9, 10,11,12]].apply(pd.to_numeric, errors='coerce')
            test_df.loc[:, [7,8, 9, 10,11,12]] = test_df.loc[:, [7,8, 9, 10,11,12]].apply(pd.to_numeric, errors='coerce')
#             print(validate_df)
            # Calculate mean for the columns
            validate_mean = validate_df[[7,8, 9, 10,11,12]].mean()
#             print("validate_mean",validate_mean)
            test_mean = test_df[[7,8, 9, 10,11,12]].mean()

            # Append results along with epoch and batch information to the results list
            results.append({
                'Epoch': epoch,
                'Batch': batch,
                'Validate_Mean_sMAPE': validate_mean[7],
                'Validate_Median_sMAPE': validate_mean[8],
                'Validate_Mean_RMSE': validate_mean[9],
                'Validate_Median_RMSE': validate_mean[10],
                'Validate_Mean_MASE': validate_mean[11],
                'Validate_Median_MASE': validate_mean[12],
                
                'Test_Mean_Mean_sMAPE': test_mean[7],
                'Test_Mean_Median_sMAPE': test_mean[8],
                'Test_Mean_Mean_RMSE': test_mean[9],
                'Test_Mean_Median_RMSE': test_mean[10],
                'Test_Mean_Mean_MASE': test_mean[11],
                'Test_Mean_Median_MASE': test_mean[12]
            })
    columns = [
                'Epoch',
                'Batch',
                'Validate_Mean_sMAPE',
                'Validate_Median_sMAPE',
                'Validate_Mean_RMSE',
                'Validate_Median_RMSE',
                'Validate_Mean_MASE',
                'Validate_Median_MASE',
                'Test_Mean_Mean_sMAPE',
                'Test_Mean_Median_sMAPE',
                'Test_Mean_Mean_RMSE',
                'Test_Mean_Median_RMSE',
                'Test_Mean_Mean_MASE',
                'Test_Mean_Median_MASE'
    ]
    result_df = pd.DataFrame(results, columns = columns)
    result_df = result_df.sort_values(by='Validate_Mean_sMAPE', ascending=True)
    result_df.to_csv("Average_Results_"+csv_file, index=None)

In [ ]:
file_list[-1]

In [45]:
#import shutil
#shutil.make_archive(OUTPUT_NAME, 'zip', DIRECTORY_TO_ZIP)